
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/46_Transformer_Block_From_Scratch.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🏗️ Transformer Block: Budujemy klocki GPT

W notatniku 24 zrozumieliśmy matematykę *Attention*.
Teraz zbudujemy pełnoprawny **Blok Transformera**.

Struktura pojedynczego bloku wygląda tak:
$$ x_{new} = x + Attention(Norm(x)) $$
$$ x_{final} = x_{new} + MLP(Norm(x_{new})) $$

Kluczowe są tu **Połączenia Rezydualne (Residual Connections)**, czyli to `x + ...`.
Działają jak autostrada dla gradientów. Dzięki nim sieć może mieć 100 warstw i nadal się uczyć (sygnał nie zanika).

Złożymy to z 3 klas:
1.  `SelfAttention` (Oczy).
2.  `FeedForward` (Mózg).
3.  `TransformerBlock` (Całość).

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Konfiguracja
EMBED_DIM = 256   # Rozmiar wektora słowa (d_model)
HEAD_DIM = 64     # Rozmiar jednej głowicy
NUM_HEADS = 4     # Ile głowic? (4 * 64 = 256, musi się zgadzać)
SEQ_LEN = 10      # Długość zdania

# Dane wejściowe (Batch=1, Słów=10, Cech=256)
x = torch.randn(1, SEQ_LEN, EMBED_DIM)

print(f"Wejście: {x.shape}")

Wejście: torch.Size([1, 10, 256])


## 1. Moduł Attention (Oczy)

Tu dzieje się magia z Query, Key, Value.
W PyTorch używamy `nn.Linear` do stworzenia wag $W_q, W_k, W_v$.

Dla uproszczenia zrobimy jedną dużą głowicę (Single Head), żeby kod był czytelny, ale w prawdziwym GPT dzieli się to na wiele mniejszych.

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.embed_dim = embed_dim
        
        # Warstwy generujące Q, K, V
        self.q_linear = nn.Linear(embed_dim, embed_dim)
        self.k_linear = nn.Linear(embed_dim, embed_dim)
        self.v_linear = nn.Linear(embed_dim, embed_dim)
        
        # Warstwa wyjściowa (łączenie wyników)
        self.out_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        # x shape: [Batch, Seq_Len, Embed_Dim]
        
        # 1. Generujemy Q, K, V
        Q = self.q_linear(x)
        K = self.k_linear(x)
        V = self.v_linear(x)
        
        # 2. Obliczamy Scores (Q * K^T)
        # transpose(-2, -1) zamienia dwa ostatnie wymiary (Seq i Dim)
        scores = torch.matmul(Q, K.transpose(-2, -1))
        
        # 3. Skalowanie (dzielimy przez pierwiastek z wymiaru)
        scores = scores / math.sqrt(self.embed_dim)
        
        # 4. Softmax (zamiana na prawdopodobieństwo)
        attention_weights = F.softmax(scores, dim=-1)
        
        # 5. Mnożymy przez V
        context = torch.matmul(attention_weights, V)
        
        # 6. Ostatnie liniowe przekształcenie
        output = self.out_linear(context)
        
        return output

# Test
attn = SelfAttention(EMBED_DIM)
out = attn(x)
print(f"Wyjście z Attention: {out.shape} (Kształt zachowany!)")

Wyjście z Attention: torch.Size([1, 10, 256]) (Kształt zachowany!)


## 2. Moduł Feed Forward (Mózg)

To jest zwykła sieć MLP włożona do środka.
Zazwyczaj rozszerza wymiar 4-krotnie (np. 256 -> 1024), a potem go zwęża z powrotem.
To tutaj model "przemyśliwuje" to, co zobaczył w Attention.

In [3]:
class FeedForward(nn.Module):
    def __init__(self, embed_dim, expansion=4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * expansion), # Rozszerzamy
            nn.ReLU(),                                   # Nieliniowość
            nn.Linear(embed_dim * expansion, embed_dim)  # Zwężamy
        )

    def forward(self, x):
        return self.net(x)

# Test
ff = FeedForward(EMBED_DIM)
out_ff = ff(x)
print(f"Wyjście z FeedForward: {out_ff.shape}")

Wyjście z FeedForward: torch.Size([1, 10, 256])


## 3. The Transformer Block (Całość)

Teraz składamy klocki.
Dodajemy dwa kluczowe elementy:
1.  **LayerNorm:** Normalizacja (żeby liczby nie uciekły w nieskończoność).
2.  **Add (Residual Connection):** Dodajemy oryginał do wyniku.

Wzór: `x = x + Layer(Norm(x))` (tzw. Pre-Norm architecture, używana w GPT-3).

In [4]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.attention = SelfAttention(embed_dim)
        self.norm1 = nn.LayerNorm(embed_dim)
        
        self.feed_forward = FeedForward(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        # KROK 1: Attention z Residualem
        # Zapisujemy kopię x (residual)
        residual = x
        # Najpierw normalizacja (Pre-Norm), potem uwaga
        x = self.norm1(x)
        x = self.attention(x)
        # Dodajemy oryginał (Add)
        x = x + residual
        
        # KROK 2: Feed Forward z Residualem
        residual = x
        x = self.norm2(x)
        x = self.feed_forward(x)
        # Dodajemy oryginał (Add)
        x = x + residual
        
        return x

# WIELKI TEST
block = TransformerBlock(EMBED_DIM)
final_output = block(x)

print("-" * 30)
print(f"Wejście: {x.shape}")
print(f"Wyjście: {final_output.shape}")
print("-" * 30)
print("Sukces! Wymiary się zgadzają.")
print("Możemy teraz postawić 96 takich bloków jeden na drugim i mamy architekturę GPT-3.")

------------------------------
Wejście: torch.Size([1, 10, 256])
Wyjście: torch.Size([1, 10, 256])
------------------------------
Sukces! Wymiary się zgadzają.
Możemy teraz postawić 96 takich bloków jeden na drugim i mamy architekturę GPT-3.


## 🧠 Podsumowanie: Autostrada dla Gradientu

Dlaczego ten "Residual Connection" (`x + residual`) jest taki ważny?

Wyobraź sobie sieć, która ma 100 warstw.
Podczas treningu (Backpropagation), gradient musi przejść przez mnożenie 100 razy.
Jeśli mnożysz liczby mniejsze od 1 (np. 0.9) przez siebie 100 razy -> wynik to 0.00002. Gradient znika. Sieć przestaje się uczyć.

**Dodawanie (`+`)** działa jak "bypass" (obwodnica).
Gradient może popłynąć "autostradą" bezpośrednio od końca do początku sieci, omijając skomplikowane obliczenia w Attention/MLP.
Dzięki temu możemy trenować **bardzo głębokie** sieci.

To jest sekret sukcesu ResNet (obrazy) i Transformerów (tekst).